In [4]:
pip install dtaidistance

  Using cached dtaidistance-2.3.12-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.9 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from datetime import timedelta
#from arcgis.features import GeoAccessor, GeoSeriesAccessor
import matplotlib.pyplot as plt
from dtaidistance import dtw
import numpy as np
from sklearn.cluster import SpectralClustering

In [16]:
apr_csv = r"principles of geocomputing/dat_202204.csv"

apr_df = pd.read_csv(apr_csv)

apr_df

,board,alight,c_load,calendar_id,block_abbr,trip_id,route_abbr,line_type,o_stop,stop_sequence_number,service_abbr,route_direction_abbr,pattern_abbr,block_stop_order,latitude,longitude,actual_arrival_time
0,1,0,3,20220401,1462,1,54,Urb Loc,11861,5,WK,W,UNDPMOA200,6,44.947217,-93.093799,11156
1,0,0,3,20220401,1462,1,54,Urb Loc,11862,6,WK,W,UNDPMOA200,7,44.946197,-93.095645,11240
2,0,0,3,20220401,1462,1,54,Urb Loc,11863,7,WK,W,UNDPMOA200,8,44.946112,-93.097146,11256
3,0,0,3,20220401,1462,1,54,Urb Loc,3264,8,WK,W,UNDPMOA200,9,44.946248,-93.099029,11268
4,0,0,3,20220401,1462,1,54,Urb Loc,47485,9,WK,W,UNDPMOA200,10,44.945413,-93.102246,11360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119328,0,0,1,20220430,1391,141,724,Sub Loc,9460,33,SAT,S,STL1BCTC00,755,45.068471,-93.319445,92244
8119329,0,0,1,20220430,1391,141,724,Sub Loc,9464,34,SAT,S,STL1BCTC00,756,45.066872,-93.319290,92256
8119330,0,0,1,20220430,1391,141,724,Sub Loc,9468,35,SAT,S,STL1BCTC00,757,45.064253,-93.319230,92276
8119331,0,0,1,20220430,1391,141,724,Sub Loc,9470,36,SAT,S,STL1BCTC00,758,45.062241,-93.319271,92292


In [17]:
apr_df = apr_df[apr_df['route_abbr'].isin([2, 3, 4, 5, 7, 9, 10])]

In [18]:
apr_df["time"] = apr_df["actual_arrival_time"].apply(lambda x: str(timedelta(seconds=x % 86400)))

apr_df

/tmp/ipykernel_287/4172815701.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apr_df["time"] = apr_df["actual_arrival_time"].apply(lambda x: str(timedelta(seconds=x % 86400)))


,board,alight,c_load,calendar_id,block_abbr,trip_id,route_abbr,line_type,o_stop,stop_sequence_number,service_abbr,route_direction_abbr,pattern_abbr,block_stop_order,latitude,longitude,actual_arrival_time,time
106,1,0,1,20220401,1103,1,5,Urb Loc,17993,1,WK,S,8SNIMOA160,3,44.975664,-93.273221,14824,4:07:04
107,0,0,1,20220401,1103,1,5,Urb Loc,17908,3,WK,S,8SNIMOA160,4,44.975172,-93.271054,14954,4:09:14
108,1,0,2,20220401,1103,1,5,Urb Loc,17910,4,WK,S,8SNIMOA160,5,44.973787,-93.268333,15088,4:11:28
109,0,0,2,20220401,1103,1,5,Urb Loc,17911,5,WK,S,8SNIMOA160,6,44.973152,-93.266697,15140,4:12:20
110,0,0,2,20220401,1103,1,5,Urb Loc,17912,6,WK,S,8SNIMOA160,7,44.972010,-93.264010,15166,4:12:46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119147,0,0,3,20220430,1250,266,3,Urb Loc,57018,65,SAT,E,WA105CED00,499,NaN,NaN,93670,2:01:10
8119148,0,2,1,20220430,1250,266,3,Urb Loc,49450,66,SAT,E,WA105CED00,500,44.950081,-93.100056,93720,2:02:00
8119149,0,0,1,20220430,1250,266,3,Urb Loc,11912,67,SAT,E,WA105CED00,501,44.950506,-93.097629,93824,2:03:44
8119150,0,1,0,20220430,1250,266,3,Urb Loc,20013,68,SAT,E,WA105CED00,502,44.947507,-93.094443,93870,2:04:30


In [19]:
# Function to round up to the next 30-minute bucket - created with assistance of ChatGPT, Open AI
def round_to_next_30_minutes(time_str):
    # Ensure time is always in HH:MM:SS format
    hours, minutes, seconds = map(int, time_str.split(':'))
    time_obj = timedelta(hours=hours, minutes=minutes, seconds=seconds)
    
    # Calculate total minutes since midnight
    total_minutes = time_obj.seconds // 60
    
    # Calculate the next 30-minute bucket
    next_bucket_minutes = (total_minutes + 29) // 30 * 30
    
    # Convert the minutes back to a timedelta object
    rounded_time = timedelta(minutes=next_bucket_minutes)
    
    # Format the rounded time as HH:MM:SS
    rounded_hours = rounded_time.seconds // 3600
    rounded_minutes = (rounded_time.seconds % 3600) // 60
    rounded_seconds = rounded_time.seconds % 60

    return f"{rounded_hours:02}:{rounded_minutes:02}:{rounded_seconds:02}"

#Apply the function to create a new column
apr_df['time_bucket'] = apr_df['time'].apply(round_to_next_30_minutes)

/tmp/ipykernel_287/4027269039.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apr_df['time_bucket'] = apr_df['time'].apply(round_to_next_30_minutes)


In [20]:
apr_wk = apr_df[apr_df['service_abbr'] == 'WK']

In [21]:
apr_wk_summ = apr_wk.groupby(["o_stop", "time_bucket"]).agg({
    "board": "sum",
    "alight": "sum",
    "latitude": "min",
    "longitude": "min"
}).reset_index()

In [22]:
#pivot table to convert to stop-based vectors - can be used to create ST clusters
apr_wk_board_pivot = pd.pivot(apr_wk_summ, values = "board", index = "o_stop", columns = "time_bucket")
apr_wk_board_pivot

time_bucket,00:00:00,00:30:00,01:00:00,01:30:00,02:00:00,02:30:00,03:00:00,04:00:00,04:30:00,05:00:00,...,19:00:00,19:30:00,20:00:00,20:30:00,21:00:00,21:30:00,22:00:00,22:30:00,23:00:00,23:30:00
o_stop,,,,,,,,,,,,,,,,,,,,,
34,28.0,23.0,1.0,22.0,NaN,NaN,NaN,NaN,13.0,NaN,...,57.0,71.0,66.0,76.0,59.0,47.0,55.0,44.0,25.0,62.0
36,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37,4.0,1.0,0.0,2.0,NaN,NaN,NaN,NaN,2.0,NaN,...,26.0,14.0,14.0,24.0,18.0,5.0,18.0,8.0,8.0,3.0
39,4.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,2.0,NaN,...,10.0,8.0,24.0,22.0,19.0,5.0,6.0,9.0,3.0,6.0
40,2.0,3.0,0.0,2.0,NaN,NaN,NaN,NaN,2.0,NaN,...,17.0,14.0,17.0,7.0,13.0,12.0,3.0,4.0,4.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80369,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0
80370,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,3.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
90236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
#fill nulls with 0s
apr_wk_board_pivot = apr_wk_board_pivot.fillna(0)

#create numpy array of values
time_series_columns = [col for col in apr_wk_board_pivot.columns if ":" in col]
time_series = apr_wk_board_pivot[time_series_columns].values

In [24]:
time_series

array([[28., 23.,  1., ..., 44., 25., 62.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 4.,  1.,  0., ...,  8.,  8.,  3.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  3.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [25]:
ds = dtw.distance_matrix_fast(time_series)

In [26]:
clustering = SpectralClustering(n_clusters = 2,
                               assign_labels = "kmeans",
                                random_state = 0).fit(ds)

/opt/conda/lib/python3.9/site-packages/sklearn/cluster/_spectral.py:589: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/manifold/_spectral_embedding.py:260: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
print(clustering)